# Imports

In [ ]:
import elasticsearch as es
from getpass import getpass
import os
import pandas as pd
import requests 
import tiktoken
from tqdm.autonotebook import tqdm
from typing import Optional

In [ ]:
pd.set_option('display.max_rows', 500)

# Get data

In [ ]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [ ]:
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
documents[0]

In [ ]:
es_client = es.Elasticsearch('http://localhost:9200') 

In [ ]:
es_client.ml?

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
query = 'How do I execute a command in a running docker container?'

In [ ]:
def elastic_search(
    query: str, 
    serp_size: int = 5,
    course_filter: Optional[str] = None 
):
    search_query = {
        "size": serp_size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }
    if course_filter is not None:
        search_query["query"]["bool"]["filter"] = {"term": {"course": course_filter}}

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs, response

In [ ]:
res, response = elastic_search(query)

In [ ]:
response['hits']['max_score']

In [ ]:
results_docs, _ = elastic_search(
    query=query,
    serp_size=3, 
    course_filter="machine-learning-zoomcamp"
)

In [ ]:
len(results_docs)

In [ ]:
results_docs[2]

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [ ]:
context = []
for result in results_docs:
    context.append(context_template.format(question=result['question'], text=result['text']))

context = "\n\n".join(context)
len(context)

In [ ]:
prompt = prompt_template.format(question=query, context=context)
len(prompt)

In [ ]:
print(prompt)

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
len(encoding.encode(prompt))

In [ ]:
code_map = pd.DataFrame(
    data=[(code, encoding.decode_single_token_bytes(code))  for code in encoding.encode(prompt)],
    columns=['code', 'token']
)

In [ ]:
code_map.iloc[:10]

In [ ]:
encoding.decode_single_token_bytes(63842)